In [1]:
#Imports
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import os.path
import random
from time import time
import string
import webbrowser
import numpy as np
import matplotlib.pyplot as plt
import string
import re
from itertools import combinations
from sys import exit

def text_between_subtrings(s, start, end):
    return s.find(start), s.find(end)

def wordify(question, max_words=30):
    question_words=question.split()
    if len(question_words)>max_words:
        question_words=question_words[:max_words]
    q=' '.join(question_words)
    #q=''.join("0" if c.isdigit() else c for c in q)
    q=re.sub('\d+', '#', q)#convert multiple digits to single digits
    q=re.sub('#.#', '#', q)#convert multiple digits to single digits
    q=re.sub(' to power of ', '^', q)
    q=re.sub('_+', '', q)
    q=re.sub('\d+\)', '', q)#these two lines get rid of any 3) _______ crap
    q = re.sub('#\)', '', q)
    if "version is this" in q:
        q = re.sub('[\bA-E]', 'V', q)  
        q = re.sub('#\)', '', q)
    else:
        "Make sure the version question reads: What version is this test"
    return q.strip()

def wordify_answers(answer, max_words=30):
    answer_words=answer.split()
    if len(answer_words)>max_words:
        answer_words=answer_words[:max_words]
    q=' '.join(answer_words)
    q=re.sub(' to power of ', '^', q)
    if len(answer_words)>1:
        q=re.sub('[\bA-E]\)', '', q)#strips the A) for all except the version question
    return q.strip()

def extract_questions(s, max_words=10):
    more='yes'
    n=1
    questionlist=[]
    while more=='yes':
        start = '\n'+str(n)
        end = '\n'+str(n+1)
        p1, p2 = text_between_subtrings(s, start, end)
        if p1!=-1 and p2!=-1:
            txt=s[p1+len(start):p2]
            ###Grab the part between ) and nA, and convert into words
            pp1, pp2 = text_between_subtrings(txt, ')', '\nA')
            txt=txt[1:txt.find('\nA')]
            txt=wordify(txt,max_words)
            questionlist.append(txt)
            n+=1
        elif p1!=-1 and p2==-1:
            txt=s[p1+len(start):]
            pp1, pp2 = text_between_subtrings(txt, ')', '\nA')
            txt=txt[1:txt.find('\nA')]
            txt=wordify(txt,max_words)
            #re.sub('[^A-Z]', '', s)
            questionlist.append(txt)
            more='no'
        else:
            print("Sum ting wong. Shouldn't go here. p1= "+str(p1)+" p2= "+str(p2))
            more='no'
    return questionlist
    

def check_uniqueness_of_QIDs(All_IDs):
    rr=range(len(All_IDs))
    combs=list(combinations(rr,2))
    for comb in combs:
        u=[x for x in All_IDs[comb[0]] if x not in All_IDs[comb[1]]]
        if u:
            print("Oops! Some question ID's are not unique: "+str(comb[0])+' and '+str(comb[1]))
            return 'n'
    return 'y'

def createQuestionIDs_and_dfs(max_words=10):
    All_IDs=[]
    dfs=[]
    for n in range(0,6):
        filename="key"+str(n)+".txt"
        if (os.path.isfile(filename)):
            with open(filename, encoding="utf-8") as file:  
                data = file.read()
            p=data.rfind('\n\n\n\n1)')
            if p!=-1: 
                questions=data[:p]
            else:
                print('Make sure questions and answers in key'+str(n)+' are separated by \n\n\n\n')
                return []
            if questions.find('version is this')==-1:
                print('Make sure there is a version question with the words "version is this". ')
                return []
            
            qids=extract_questions(questions, max_words)
            qnum=len(qids)
            np_qids=np.array(qids).reshape(qnum,1)
            All_IDs.append(qids)
            
            
            aa=extract_choices(questions, qnum)
            np_aa=np.array(aa).reshape(qnum,len(aa[0]))
            v1=np.concatenate((np_qids,np_aa),axis=1)
            df=pd.DataFrame(v1,columns=["Question","A","B","C","D","E"],index=range(1,qnum+1))
            df['Blank']=''
            dfs.append(df)
            
            
    if check_uniqueness_of_QIDs(All_IDs)=='y':
        return {"IDs":All_IDs, "dfs":dfs}
    else:
        print("QIDs not unique.")
        return {"IDs":[], "dfs":[]}

def extract_choices(s, qnum):
    all_choices=[]
    for n in range(1,numberOfQuestions+1):
        txt=''
        start = '\n'+str(n)
        end = '\n'+str(n+1)
        p1, p2 = text_between_subtrings(s, start, end)
        if p1!=-1 and p2!=-1:
            txt=s[p1+len(start):p2]
        if p1!=-1 and p2==-1:
            txt=s[p1+len(start):]
        anslist=[]
        for m in range(0,4):
            start=invkeydictionary[m]+')'
            end=invkeydictionary[m+1]+')'
            #looking for answer choices as the text between A) and B) for example
            p1=txt.rfind(start)
            p2=txt.rfind(end)
            if p2!=-1:
                ans=txt[p1:p2].strip()
            else:
                ans=txt[p1:].strip()
            #if p2 was not found, do txt[p1:] rather than txt[p1:-1] which would truncate the last letter
            ans=wordify_answers(ans)
            if ans:
                anslist.append(ans)
            else:
                anslist.append('')
        if p2!=-1:
            ans=txt[p2:].strip()
            #ans=re.sub(' to power of ', '^', ans)
            ans=wordify_answers(ans)
            anslist.append(ans)
        else:
            anslist.append('')
        all_choices.append(anslist)
    return all_choices


def getAllKeys(fixed_points_per_question=1):
    '''read all files named key0-key4, make them into strings and and return a dictionary
    containing a list of keys and also the number of questions'''
    keylist=[]
    pointlist=[]
    for n in range(0,6):
        filename="key"+str(n)+".txt"
        if (os.path.isfile(filename)):
            with open(filename, encoding="utf-8") as file: 
                data = file.read()
            u=re.search('\n\n\n\n+1\)',data)
            if u is not None:
                answers=data[u.start():].strip()
            else:
                print('Make sure questions and answers in key'+str(n)+' are separated by \n\n\n\n')
                return None

            ukey=re.findall('[\bA-E]',answers)
            ukey=[str(ord(x)-65) for x in ukey]
            key=''.join(ukey)
            keylist.append(key)
            
            qnum=len(key)

            #This assumes that the answerkey contains points EITHER in the user-supplied form 8) A /tab 3/n OR testgen-supplied form "Points: 3"
            upoints=re.findall('\t\d+',answers)+re.findall(':\s\d+',answers)
            if upoints:
                _=''.join(upoints)
                points=re.sub('[^ \d+]',' ', _).split()
                points=[int(x) for x in points ]
                assert(len(points)==qnum)
            else:
                points=[fixed_points_per_question]*qnum
            
            points[-1]=0
            pointlist.append(points)
    
    if not keylist:
        print("Answerkeys not found. Make sure they are named key0.txt, key1.txt etc")
    return {"keylist":keylist,"pointlist":pointlist,"numberOfQuestions":qnum}

def process_grades(data,outs, QIDs,analysis=False):
    '''grades all exams using correct keys, writes questions missed and scores'''

    for NN in range(0, data.shape[0]):
        #print("NN: "+str(NN))
        ans=data.iat[NN,2]
        if len(ans)==numberOfQuestions-1:
            v=guess_the_version(ans)
            ans=ans+str(v)
            print("Assuming version "+invkeydictionary[v]+ " for: "+data.iat[NN,1]+" (Srl No: "+data.iat[NN,0]+").")
        check1=gradeWithKeylist(ans, outs, QIDs, analysis, NN)
        data.iat[NN,3]=check1['missed']
        data.iat[NN,4]=check1['score']
        data.iat[NN,5]=100.0*float(check1['score'])/float(new_totalpoints)

    if analysis:
        #remove the column with the version numbers
        ####TO DO: find the version column rather than assume its the last one?
        analysis_df.drop(analysis_df.columns[analysis_df.shape[1]-2], axis=1, inplace=True)
    return data


def gradeWithKeylist(ans,outs, QIDs, analysis=False, N=0):
    '''multiple versions - find the correct key as indicated on the last question on the exam '''
    keylist=outs["keylist"]
    pointlist=outs["pointlist"]
    numberOfQuestions=outs["numberOfQuestions"] 
    
    assert(keylist!=[])
    assert (ans[-1:] in ['0','1','2','3','4'])
    whichKey=int(ans[-1:])
    
    key=keylist[whichKey]
    points=pointlist[whichKey]
    assert len(key)==len(ans)
    missed=""
    rejalt=[1]*numberOfQuestions
    for n in range(0,len(key)-1):
        if key[n]!=ans[n]:
            #print("storing")
            missed+=str(n+1)+", "
            rejalt[n]=0
    if sum(rejalt)==numberOfQuestions:
        missed="v"+invkeydictionary[whichKey]+": "+"ALL CORRECT"
    else:
        missed="v"+invkeydictionary[whichKey]+": "+missed[:len(missed)-2]
    score=sum([i*j for i,j in zip(points,rejalt)])
    

    mydict1 = dict(zip(QIDs[whichKey],rejalt))
    #sortedIDs=sorted(mydict1.keys())
    sorted_rejalt=[mydict1[k] for k in QIDs[0]]####its sorted according to v0
    

    
    if analysis:   
        sorted_rejalt.append(score)
        analysis_df.loc[N] = sorted_rejalt 
        
        mydict2 = dict(zip(QIDs[whichKey],ans))  
        #sortedIDs=sorted(mydict2.keys())
        sorted_ans=[mydict2[k] for k in QIDs[0]]
        allAnswers_df.loc[N] = sorted_ans
    

        
    return {'missed':missed, 'score':score}


def guess_the_version(no_version):
    scores=[]
    for n in range(numberOfVersions):
        try_version=no_version+str(n)
        check1=gradeWithKeylist(try_version, outs, QIDs, analysis=False)
        scores.append(check1['score'])
    return scores.index(max(scores))



def make_into_percent(adf):
    '''Takes a dataframe and converts into np array to perform math operations.
    In this case it divides each element by the sum of the row and times by 100 to make it a percent.'''
    cols=list(adf)
    #adf=adf.applymap(float)
    np_df=adf.values
    np_df=100*np_df/np.sum(np_df, axis=1, keepdims=True)
    return pd.DataFrame(np_df,columns=cols)


def mark_correct_answers(pardf,n,keylist):
    
    key_to_use=keylist[n]
    for m in range(0,pardf.shape[0]):
        cor=int(key_to_use[m])
        pardf.iloc[m,cor]='[Correct] '+pardf.iloc[m,cor]
    return pardf

def analyse_items(a_df, koschen_paper, keylist):
    '''create separate dfs with how many marked correct for each version separately'''
    vers=['0','1','2','3','4']
    lvers=['A','B','C','D','E']
    outvars=[]
    versionsfound=[]
    students_in_this_version=[]

    for n in range(0,len(koschen_paper)):
        ch=str(n)
        ##check for "which version..." to find the version and see if it matches ch
        ver_qnum=koschen_paper[n].index[koschen_paper[n]['Question'].str.contains("version is this")].tolist()[0]-1
        #return the indexes of the cells which contain the given text, as a list
        ver_q=koschen_paper[n].iloc[ver_qnum,0]
        part_df = a_df.loc[a_df[ver_q]==ch]
        if not part_df.empty:
            print("Analysing data for version: "+lvers[n])
            part_df=part_df.T
            part_df=part_df.apply(pd.Series.value_counts, axis=1).fillna(0)
            #part_df=addStarsToCorrectChoices(part_df,keylist,n, QIDs)
            if ' ' in list(part_df):
                part_df.rename(columns={' ': 'Blank'}, inplace=True)   
            for pp in vers:
                if pp in list(part_df):
                    part_df.rename(columns={pp: lvers[int(pp)]}, inplace=True)
            ####add back missing columns and rearrange in alphabetical order
            cols_present=list(part_df)
            allcols=['A','B','C','D','E','Blank']
            cols_missing=[x for x in allcols if x not in cols_present]
            if cols_missing:
                for x in cols_missing:
                    part_df[x]=0.0
            part_df = part_df[allcols]
            part_df = part_df.reindex(koschen_paper[n]["Question"])#This rearranges the questions as per the correct version
            num_students=part_df.iloc[0,:].sum()
            part_df=make_into_percent(part_df)
            part_df=part_df.applymap('{:,.0f}%'.format)

            kp=koschen_paper[n].copy()
            kp.set_index("Question", inplace=True)
            kp.index.name=None ###This prevents an extra fictitious row from showing up in the header row
            indexlist=kp.index.tolist()
            part_df.index = indexlist
            part_df=kp+"<:"+part_df+":>"
            part_df=part_df.applymap(lambda m: re.sub('<:0%:>', '', m) )
            
            part_df=mark_correct_answers(part_df,n,keylist)
            
            
            outvars.append(part_df)
            versionsfound.append(lvers[n])
            students_in_this_version.append(num_students)
    
    
#     Return two dict so that the html function knows which version each table belongs to, and how many students took that version    
    return dict(zip(versionsfound,outvars)), dict(zip(versionsfound,students_in_this_version))


    
def make_item_analysis(a_df):
    
    item_analysis_df=pd.DataFrame(index=["Difficulty","Discrimination"],columns = list(analysis_df))
    n=a_df.shape[0]
    v=analysis_df.values#converts into np array
    
    
    #Calculating Difficulty
    u1=np.sum(v,axis=0)/n
    item_analysis_df.loc["Difficulty"]=u1[0:u1.shape[0]]
    
    #Calculating Discrimination
    students_per_group=n//3
    if students_per_group<2:
        print("Need more students for discrimination analysis (at least 7).")
    else:
        v=v[0:v.shape[0]-1,:]#exclude the bottom row which had the sums (total number correct for each q)
        v=v[v[:,v.shape[1]-1].argsort()[::-1]]#sort descending by scores
        #print(v)
        u1=np.sum(v[0:students_per_group,:],axis=0)#top scorers
        u2=np.sum(v[n-students_per_group : n,:],axis=0)#bottom scorers
        disc=(u1-u2)/students_per_group
        disc.shape
        item_analysis_df.loc["Discrimination"]=disc[0:disc.shape[0]]
    
    item_analysis_df.drop("score", axis=1, inplace=True)
    return item_analysis_df.T

#################Writing data#############################
def write_to_xl(adf, rfilename):
    adf.rename(columns={'Srl No': 'Serial Number Text Grade <Text>'})#This facilitates the Vlookup later
    writer = pd.ExcelWriter(rfilename+'_processed.xlsx')
    adf.to_excel(writer,'Sheet1',index=False)
    writer.save()
    
def write_to_csv(adf, rfilename):
    filename = rfilename+'_processed.csv'
    #adf=adf.rename(columns={'Srl No': 'Serial Number Text Grade <Text>'})#This facilitates the Vlookup later
    adf.to_csv(filename, index=False)
    

def write_to_webpage(j_df, ia_df, pointdrop, ncbv={}, sbv={},  histobin=11):
    filename = rawdatafilename+'_summary.html'
    f = open(filename,'w',encoding='utf8')
    
    max_score=max(j_df['Score'])
    number_of_maxes=len([x for x in j_df['Score'] if x==max_score])
    
    uu=j_df['Percentage'].describe().to_frame()
    uu.rename(columns={'Percentage': "Value"}, index={'count':'Number of Students','mean':'Mean(%)', 'std':'Standard Deviation', 'min':'Lowest(%)','25%':'25th percentile', '50%':'50th percentile', '75%':'75th percentile', 'max':"Highest(%)"},inplace=True)
    uu.loc['Maximum Available(%) '] = [100*totalpoints/new_totalpoints]
    uu.loc['Number of Top Scorers'] = number_of_maxes
    uu.loc['Points dropped '] = pointdrop

    pre="<h2>Summary Data:</h2>"
    summary_table=uu.to_html(float_format=lambda x: '%10.2f' % x).replace("dataframe" ,"sumdat")
    summary_table=pre+summary_table
    post="<p>Score Distribution:</p>"
    
    figure = plt.figure()
    df['Percentage'].plot(kind='hist', bins=10)
    plt.xlim(xmax=100*totalpoints/new_totalpoints)
    figure.savefig('histo.svg')
    
    pre="<h2>Score Distribution:</h2>"
    image_code='<img src="histo.svg" alt="histogram">'
    image=pre+image_code

    pre="<h2>Item Analysis:</h2>"
    item_analysis_table=ia_df.sort_values('Difficulty').to_html(float_format=lambda x: '%10.2f' % x).replace("dataframe" ,"customers")
    item_analysis_table=pre+item_analysis_table

    missed_tables=""
    if ncbv:
        for ver in ['A', 'B','C','D','E']:
            if ver in ncbv.keys():
                if sbv:
                    nn=str(int(sbv[ver]))
                    pre="<h2>Distractor Analysis for Version "+ver+": ("+nn+" Students) </h2>"
                else:
                    pre="<h2>Distractor Analysis for Version "+ver+":</h2>"
                post=ncbv[ver].to_html()
                post=post.replace("dataframe" ,"howmany")
                post=post.replace('<td>[Correct]','<td class="correct">')
                post=post.replace('&lt;:','<span class="answered_by">')
                post=post.replace(':&gt;','</span>')
                missed_tables+=pre+post

    ##insert a script to make the webpage latex compatible. Edit the final  html file as \( \latex\code \)
    html_start = """<html>
    <head>
     <link rel="stylesheet" type="text/css" href="mystyle.css">
    <script type="text/javascript" async
    src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.5/latest.js?config=TeX-MML-AM_CHTML" async>
    </script>
    </head>
    <body>
    <h1>Data Analysis</h1>
    """


    html_end='''</body>
    </html>'''
    message=html_start+summary_table+image+item_analysis_table+missed_tables+html_end
    f.write(message)
    f.close()

    #Change path to reflect file location
    webbrowser.open_new_tab(filename)

###############################################

############### Checking for incorrect Srl Nos################


def check_serial_numbers(gradebook,datafile):
    df=pd.read_csv(gradebook, usecols=["Last Name", "First Name", "Serial Number Text Grade <Text>"])
    df_all=pd.read_excel(datafile, header=None, parse_cols=1,names = ["Serial Number Text Grade <Text>", "Name"])

    checker_left = pd.merge(df,df_all[['Serial Number Text Grade <Text>','Name']],on='Serial Number Text Grade <Text>', how='left')
    checker_right = pd.merge(df,df_all[['Serial Number Text Grade <Text>','Name']],on='Serial Number Text Grade <Text>', how='right')
    u1=checker_left[checker_left.isnull().any(axis=1)]
    u2=checker_left[checker_left['Serial Number Text Grade <Text>'].duplicated(keep=False)]
    u3=checker_right[checker_right.isnull().any(axis=1)]
    error_rpt=pd.concat([u1, u2, u3], axis=0)
    error_rpt["Last Name"] = error_rpt["Last Name"]+' '+error_rpt['First Name']
    error_rpt.drop('First Name', axis=1,inplace=True)
    error_rpt.rename(columns={'Serial Number Text Grade <Text>': 'Srl No', 'Last Name':'Registered Name', 'Name':'Entered Name'}, inplace=True) 
    return error_rpt

Manually convert the dat file into an excel file using excel. Only extract the serial number,
name and Answers, and make sure Answers is text.

In [3]:
#COLLECTING DATA
rawdatafilename='All'
datafilename=rawdatafilename+'.xlsx'
gradebook='BSGrades.csv'
if not (os.path.isfile(datafilename)) or not (os.path.isfile(gradebook)):
    print('Make sure All.xlsx and BSGrades.csv are ready first')
    exit("Necessary files not found")
xls_file = pd.ExcelFile(datafilename)
df = xls_file.parse('Sheet1', header=None, parse_cols=2,names = ["Srl No", "Name", "Answers"], dtype='str')
#parse_cols makes sure that only cols 0,1 and 2 are extracted
#checking for blanks, print only if blanks found
if not df[df['Answers'].str.contains(" ")].empty:
    blankers=df[df['Answers'].str.contains(" ")]
    display(blankers)
ER=check_serial_numbers(gradebook,datafilename)
if not ER.empty:
    display(ER)

df["Missed"] = ""
df["Score"]=0
df["Percentage"]=np.nan

,Registered Name,Srl No,Entered Name
1,Aburto Julio Luis,1003,NaN
91,Raman Rohith,1055,NaN
127,Zong zongzong,1133,NaN


In [4]:
#initializing everything
keydictionary={"A":"0","B":"1","C":"2","D":"3","E":"4"}
invkeydictionary={0:"A",1:"B",2:"C",3:"D",4:"E"}
points_per_question=3
point_drop=2
outs = getAllKeys(points_per_question)
keylist=outs["keylist"]
pointlist=outs["pointlist"]
numberOfQuestions=outs["numberOfQuestions"]
totalpoints=sum(pointlist[2])
new_totalpoints=totalpoints-point_drop
numberOfVersions=len(keylist)
numberOfStudents=df.shape[0]
quids_dfs=createQuestionIDs_and_dfs(max_words=50)
QIDs=quids_dfs["IDs"]
koschen_paper=quids_dfs['dfs']
if not keylist or not pointlist or numberOfQuestions==0 or not QIDs or not koschen_paper:
    exit("Something went wrong in importing the questions")
headings=QIDs[0].copy()#####headings=QIDs[0] assigns by reference, so changing headings will change QIDs[0]
allAnswers_df = pd.DataFrame(index=range(numberOfStudents), columns = headings)# stores all student answers for each question, questions are the columns
headings.append("score")
analysis_df  = pd.DataFrame(index=range(numberOfStudents),columns = headings)# stores whether answer was correct, questions are the columns, last column is the score


In [5]:
##Everything initialized. Running the code now
didnt_enter_versions=df[df['Answers'].map(len)!=numberOfQuestions]
if not didnt_enter_versions.empty:
    didnt_enter_versions=didnt_enter_versions[['Srl No', 'Name','Answers']]
    display(didnt_enter_versions)
print("Starting calculations...")
starttime = time()
df=process_grades(df,outs, QIDs, analysis=True)
number_correct_by_version, students_by_version = analyse_items(allAnswers_df, koschen_paper,keylist)
item_analysis_df = make_item_analysis(analysis_df) 
endtime = time()
print("Done. That took: "+str(endtime-starttime)+ " sec for "+str(numberOfStudents)+" students")
#check for duplicated serial numbers
dupes=df[df['Srl No'].duplicated(keep=False)]
if not dupes.empty:
    print('Warning: Duplicates in serial numbers found!')
    display(dupes)    
max_score=analysis_df.at[analysis_df['score'].idxmax(),'score']

Starting calculations...
Analysing data for version: A
Analysing data for version: B
Analysing data for version: C
Analysing data for version: D
Done. That took: 0.2772691249847412 sec for 126 students


In [8]:
write_to_csv(df.sort_values('Score', ascending=False),rawdatafilename)
if (os.path.isfile('mystyle.css')):
    write_to_webpage(df, item_analysis_df, point_drop, number_correct_by_version, students_by_version)
else:
    print("Copy the css file over first")

In [ ]:
#Grades and displays the N'th entry in the list using a keylist
N=random.randint(1,df.shape[0]-1)
#for N in range(df.shape[0]-1):
#N=13
check1=gradeWithKeylist(df.iat[N,2], outs, QIDs, analysis=False)
print("Row "+ str(N)+": "+df.iat[N,1]+", "+str(df.iat[N,0])+". Missed "+str(check1['missed'])+ ". Scored "+str(check1['score'])+"/"+str(totalpoints))
print('That is: '+str(100*check1['score']/new_totalpoints)+'% with '+str(point_drop)+' points dropped')

In [ ]:
###################### DEV AREA###########################################


# filename="key"+str(0)+".txt"
# if (os.path.isfile(filename)):
#     with open(filename, encoding="utf-8") as file:  
#         data = file.read()
#     p=data.rfind('\n\n\n\n1)')
# s=data[:p]

# n=25
# start = '\n'+str(n)
# end = '\n'+str(n+1)
# p1, p2 = text_between_subtrings(s, start, end)
# if p1!=-1 and p2!=-1:
#     txt=s[p1+len(start):p2]
# if p1!=-1 and p2==-1:
#     txt=s[p1+len(start):]

# print(txt)
# anslist=[]
# for m in range(0,4):
#     start=invkeydictionary[m]+')'
#     end=invkeydictionary[m+1]+')'
#     #looking for answer choices as the text between A) and B) for example
#     p1=txt.rfind(start)
#     p2=txt.rfind(end)
#     if p2!=-1:
#         ans=txt[p1:p2].strip()
#     else:
#         ans=txt[p1:].strip()
#     #if p2 was not found, do txt[p1:] rather than txt[p1:-1] which would truncate the last letter
#     ans=wordify_answers(ans)
#     if ans:
#         print('p1= '+str(p1)+', p2= '+str(p2)+" ans="+ans+' z1')
#         anslist.append(ans)
#     else:
#         print('p1= '+str(p1)+', p2= '+str(p2)+" ans= None"+' z1')
#         anslist.append('')
# if p2!=-1:
#     ans=txt[p2:].strip()
#     #ans=re.sub(' to power of ', '^', ans)
#     ans=wordify_answers(ans)
#     print('p1= '+str(p1)+', p2= '+str(p2)+" ans="+ans+' z2')
#     anslist.append(ans)
# else:
#     print('p1= '+str(p1)+', p2= '+str(p2)+" ans= None"+' z3')
#     anslist.append('')



vers=['0','1','2','3','4']
lvers=['A','B','C','D','E']
n=2
a_df=allAnswers_df
ch=str(n)
##check for "which version..." to find the version and see if it matches ch
ver_qnum=koschen_paper[n].index[koschen_paper[n]['Question'].str.contains("version is this")].tolist()[0]-1
#return the indexes of the cells which contain the given text, as a list
ver_q=koschen_paper[n].iloc[ver_qnum,0]
part_df = a_df.loc[a_df[ver_q]==ch]
if not part_df.empty:
    print("Analysing data for version: "+lvers[n])
    part_df=part_df.T
    part_df=part_df.apply(pd.Series.value_counts, axis=1).fillna(0)
    #part_df=addStarsToCorrectChoices(part_df,keylist,n, QIDs)
    if ' ' in list(part_df):
        part_df.rename(columns={' ': 'Blank'}, inplace=True)   
    for pp in vers:
        if pp in list(part_df):
            part_df.rename(columns={pp: lvers[int(pp)]}, inplace=True)
    ####add back missing columns and rearrange in alphabetical order
    cols_present=list(part_df)
    allcols=['A','B','C','D','E','Blank']
    cols_missing=[x for x in allcols if x not in cols_present]
    if cols_missing:
        for x in cols_missing:
            part_df[x]=0.0
    part_df = part_df[allcols]
    part_df = part_df.reindex(koschen_paper[n]["Question"])
    num_students_this_version=part_df.iloc[0,:].sum()
    part_df=make_into_percent(part_df)
    part_df=part_df.applymap('{:,.0f}%'.format)

    kp=koschen_paper[n].copy()
    kp.set_index("Question", inplace=True)
    kp.index.name=None ###This prevents an extra fictitious row from showing up in the header row
    indexlist=kp.index.tolist()
    part_df.index = indexlist
    part_df=kp+"<"+part_df+">"
    part_df=part_df.applymap(lambda m: re.sub('<0%>', '', m) )


In [ ]:
part_df

In [ ]:
part_df.reindex(koschen_paper[2]["Question"])

In [ ]:
part_df.reindex

In [ ]:
post1=post.replace('<td>[Correct]','<td class="correct">')

In [ ]:
post1

In [ ]:
max_score=max(df['Score'])
number_of_maxes=len([x for x in df['Score'] if x==max_score])

In [ ]:
[i for i, x in enumerate(df['Score']) if x == 42]

In [9]:
list(df)

['Srl No', 'Name', 'Answers', 'Missed', 'Score', 'Percentage']

In [32]:

###############An attempt to develop a match finder

# match_threshold=int(0.9*numberOfQuestions)
# match_df=pd.DataFrame(columns=["Student 1", "Student 2", "Matches"])

# n=22
# m=54
# a1=df.loc[n,'Answers']
# a2=df.loc[m, 'Answers']
# matches=[ele for count,ele in enumerate(a1) if a2[count]==ele]
# match_length=len(matches)
# if match_length >= match_threshold:
#     nn=len(match_df.index)
#     match_df.loc[0, "Student 1"]=df.loc[n,'Name']+" ("+df.loc[n,'Srl No']+')'
#     match_df.loc[0, "Student 2"]=df.loc[m,'Name']+" ("+df.loc[m,'Srl No']+')'
#     match_df.loc[0, "Matches"]=matches
    
    